<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/6-doc2query/6_t5%2Bdoc2query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6- T5 + doc2query
Júlia Ferreira Tessler

In [ ]:
!pip install transformers -q
!pip install ftfy -q
!pip install sentencepiece -q
!pip install evaluate -q
!pip install sacrebleu -q
!pip install pyserini -q
!pip install faiss-cpu -q
!pip install trectools -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
workdir = '/content/gdrive/MyDrive/Unicamp/Deep Learning Aplicado à Sistemas de Busca/Notebooks/'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 12 21:53:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import pandas as pd
import numpy as np
import ftfy
import evaluate
import json

from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
    T5Tokenizer,
    T5ForConditionalGeneration
)
from transformers.optimization import Adafactor, AdafactorSchedule
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

## Downloading and preparing dataset

In [ ]:
# Download training data
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2023-04-10 22:20:48--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 74.125.137.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.1s    

2023-04-10 22:20:48 (61.0 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



According to ChatGPT:

The MSMARCO Triples Tiny dataset is a smaller subset of the full MSMARCO Triples dataset that contains 3-tuples of (query, positive document, negative document) pairs. Each triple in the MSMARCO Triples Tiny dataset consists of the following fields:

* Query: The search query that was issued by the user.
* Positive Document: A document that is relevant to the query and is considered as a positive example.
* Negative Document: A document that is irrelevant to the query and is considered as a negative example.

All three fields are represented as strings in the dataset. The MSMARCO Triples Tiny dataset is designed to be used for training and evaluating machine learning models for query-document ranking and relevance prediction tasks. The dataset is smaller than the full MSMARCO Triples dataset, containing only 31,000 triples, but can be used for quick experimentation and prototyping of machine learning models.

In [ ]:
# Code from ChatGPT
df = pd.read_csv('msmarco_triples.train.tiny.tsv', 
                 delimiter = '\t', 
                 header = None, 
                 names = ['query', 'pos_doc', 'neg_doc'])

df.head()

,query,pos_doc,neg_doc
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


Cleaning for weird chars in text. This idea (and code!) came from Marcos Piau's notebook (:

In [ ]:
df['query'] = df['query'].apply(ftfy.fix_text)
df['pos_doc'] = df['pos_doc'].apply(ftfy.fix_text)
df.drop('neg_doc', inplace = True, axis = 1)

df.head()

,query,pos_doc
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,..."


### Separate into train and test data

In [ ]:
X = df['pos_doc'].tolist()
y = df['query'].tolist()

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = 1000, random_state = 42)

## Get the model

In [ ]:
model_name = 't5-base'

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Training
We should use sacreBLEU to evaluate the model after some steps.

In [ ]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

### Tokenize dataset

In [ ]:
class Doc2QueryDataset(Dataset):
  def __init__(self, X, Y, tokenizer):
    self.tokenizer = tokenizer
    self.X = X
    self.Y = Y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):    
    tokenized_input = self.tokenizer(self.X[index])
    tokenized_query = self.tokenizer(self.Y[index])
    return {"input_ids": tokenized_input["input_ids"], 
            "attention_mask": tokenized_input["attention_mask"], 
            "labels": tokenized_query["input_ids"]}

In [ ]:
train_dataset = Doc2QueryDataset(X_train, y_train, tokenizer)
val_dataset = Doc2QueryDataset(X_eval, y_eval, tokenizer)

In [ ]:
batch_size = 32
steps = 50
epochs = 20

In [ ]:
# Got this chunk of code from Monique (https://github.com/monilouise/unicamp-P_IA368DD_2023S1/blob/main/Aula_6/T5_%2B_doc2query_finetuning.md)
print("batch size = ", batch_size)
print("len train_dataset = ", len(train_dataset))

training_args = Seq2SeqTrainingArguments(output_dir=f"{workdir}/doc2query",
                                          overwrite_output_dir = True,
                                          per_device_train_batch_size = batch_size,
                                          per_device_eval_batch_size = batch_size,
                                          gradient_accumulation_steps = 8,
                                          evaluation_strategy = 'steps',
                                          eval_steps = steps, 
                                         logging_steps = steps, 
                                          save_steps = steps,
                                          predict_with_generate = True,
                                          fp16 = False,
                                          bf16 = True,
                                          num_train_epochs = epochs,
                                          load_best_model_at_end = True,
                                          metric_for_best_model = 'bleu',
                                          save_total_limit = 2
                                        )

#If you use mixed precision, you need all your tensors to have dimensions that are multiple of 8s to maximize the benefits of your tensor cores.
#So pas_to_multiple_of=8 is a good value
#Ref.: https://discuss.huggingface.co/t/whats-a-good-value-for-pad-to-multiple-of/1481


batch size =  32
len train_dataset =  10000


In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model = model,
    label_pad_token_id = -100,
    pad_to_multiple_of = 8 if training_args.fp16 else None,
)

In [ ]:
# optimizer = Adafactor(model.parameters(), 
#                       lr = 1e-5, #1e-3 fez com que a val loss e o sacrebleu não reduzissem
#                       scale_parameter = False, 
#                       relative_step = False, 
#                       warmup_init = False
#                       )
# lr_scheduler = AdafactorSchedule(optimizer)

# Monique's way
optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True)
lr_scheduler = AdafactorSchedule(optimizer)

## Training

In [ ]:
trainer = Seq2SeqTrainer(model = model,
                        args = training_args,
                        train_dataset = train_dataset,
                        eval_dataset = val_dataset,
                        data_collator = data_collator,
                        tokenizer = tokenizer,
                        compute_metrics = compute_metrics,
                        optimizers = (optimizer, lr_scheduler)
                        )

In [ ]:
train_results = trainer.train()

Step,Training Loss,Validation Loss,Bleu
50,5.117800,4.094290,1.232941
100,3.031000,1.983261,8.405212
150,2.011700,1.699693,14.273676
200,1.813100,1.609647,15.646456
250,1.683900,1.555577,17.093088
300,1.593600,1.517584,18.752896
350,1.494900,1.495356,19.137875
400,1.410900,1.484691,19.543947
450,1.325600,1.481513,19.733829
500,1.249000,1.474862,19.887115


In [ ]:
trainer.save_model(workdir+'t5_model')
metrics = trainer.evaluate()
with open(f"{workdir}/doc2query/metrics.json", 'w') as f:
  json.dump(metrics,f)

In [ ]:
metrics

{'eval_loss': 1.5980764627456665,
 'eval_bleu': 20.553625296051234,
 'eval_runtime': 28.843,
 'eval_samples_per_second': 34.67,
 'eval_steps_per_second': 1.109,
 'epoch': 19.94}

## TREC-COVID - Baseline BM25

For this part, I'd like to thank Marcus Borela, for the useful insights.

### Index preparation

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-04-12 22:02:21--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.227.219.41, 13.227.219.105, 13.227.219.125, ...
Connecting to huggingface.co (huggingface.co)|13.227.219.41|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1681596141&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l

In [ ]:
!mkdir trec-covid
!mv queries.jsonl.gz trec-covid/

mv: cannot stat 'queries.jsonl.gz': No such file or directory


In [ ]:
!gunzip trec-covid/queries.jsonl.gz

In [ ]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
from evaluate import load
from tqdm import tqdm

In [ ]:
searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:08, 27.0MB/s]                           


In [ ]:
index_reader = IndexReader.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
index_reader.stats()

{'total_terms': 20822821,
 'documents': 171331,
 'non_empty_documents': 171331,
 'unique_terms': 202648}

### Dataset preparation

In [ ]:
with open('trec-covid/queries.tsv','w') as f_out:
  with open('trec-covid/queries.jsonl', 'r') as f:
    for line in f:
      data = json.loads(line)
      id = data['_id']
      text = data['text']
      f_out.write(f'{id}\t{text}\n')

In [ ]:
!mkdir runs/
run = 'runs/run.trec-covid.bm25.txt'

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

!mv test.tsv trec-covid/

--2023-04-12 22:02:08--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 13.227.219.41, 13.227.219.105, 13.227.219.125, ...
Connecting to huggingface.co (huggingface.co)|13.227.219.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.09s   

2023-04-12 22:02:08 (10.9 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
topics = pd.read_csv("trec-covid/queries.tsv", sep="\t", header=None, 
                      names=["_id", "query"])
topics.drop(columns = {'_id'}, inplace = True)
topics.head()

,query
0,what is the origin of COVID-19
1,how does the coronavirus respond to changes in...
2,will SARS-CoV2 infected people develop immunit...
3,what causes death from Covid-19?
4,what drugs have been active against SARS-CoV o...


### Searching

In [ ]:
# From Marcus Borela
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
  Usada no notebook da aula 2

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
    cnt = 0
    for id in tqdm(range(0, len(topics['query'])), desc='Running Queries'):
        # print(f'id = {id}')
        query = topics['query'][id]
        # print(f'query = {query}')

        hits = searcher.search(query, num_max_hits)
        for i in range(0, len(hits)):
            _ = runfile.write(f'{id+1} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} SemExpansao\n')
            # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt} queries completed')

In [ ]:
run_all_queries(run, topics, searcher, 1000)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [00:04<00:00, 11.52it/s]


In [ ]:
run = pd.read_csv(run, sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run.head()

,query,q0,docid,rank,score,system
0,1,Q0,dv9m19yk,1,7.285900,SemExpansao
1,1,Q0,0paafp5j,2,6.214600,SemExpansao
2,1,Q0,96zsd27n,3,6.214599,SemExpansao
3,1,Q0,hmvo5b0q,4,6.163500,SemExpansao
4,1,Q0,1ij25a7u,5,5.838400,SemExpansao


In [ ]:
run = run.to_dict(orient="list")

In [ ]:
qrel = pd.read_csv("trec-covid/test.tsv", sep="\t", header = None, 
                   skiprows = 1, names=["query", "docid", "rel"])
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [ ]:
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [ ]:
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions = [run], references = [qrel])

In [ ]:
results

{'runid': 'SemExpansao',
 'num_ret': 50000,
 'num_rel': 24673,
 'num_rel_ret': 9605,
 'num_q': 50,
 'map': 0.18712850250314475,
 'gm_map': 0.1248255966783779,
 'bpref': 0.32940840203100064,
 'Rprec': 0.28470678646102393,
 'recip_rank': 0.8528571428571429,
 'P@5': 0.6880000000000002,
 'P@10': 0.644,
 'P@15': 0.608,
 'P@20': 0.5989999999999999,
 'P@30': 0.5753333333333334,
 'P@100': 0.4718,
 'P@200': 0.39480000000000004,
 'P@500': 0.27664,
 'P@1000': 0.19210000000000005,
 'NDCG@5': 0.6323889863121053,
 'NDCG@10': 0.5946917010118077,
 'NDCG@15': 0.561630324213359,
 'NDCG@20': 0.5463332676967534,
 'NDCG@30': 0.5233190162754622,
 'NDCG@100': 0.43823617750837557,
 'NDCG@200': 0.38302424232653026,
 'NDCG@500': 0.3554494781191315,
 'NDCG@1000': 0.405955738412841}

## Document Expansion

### Get model

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained(f"{workdir}t5_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def predict(texts, do_sample=False, num_beams=1, num_return_sequences=1):
  input_ids = tokenizer(texts, return_tensors="pt", 
                        padding="max_length", truncation=True).input_ids.to(device)
  sequence_ids = model.generate(input_ids, do_sample=do_sample,
                                num_beams=num_beams, max_new_tokens=128, 
                                num_return_sequences=num_return_sequences)
  sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
  return sequences

### Get corpus to retrieve queries

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
!mv corpus.jsonl.gz trec-covid/
!gunzip trec-covid/corpus.jsonl.gz

--2023-04-12 11:23:23--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.224.249.63, 13.224.249.86, 13.224.249.89, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.63|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1681557803&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9u

peguei todo esse trecho final da monique, vamos ver se consigo trabalhar em cima disso

In [ ]:
corpus = []

with open("trec-covid/corpus.jsonl", 'r', encoding='utf-8') as f:
    # Read the file line by line
    lines = f.readlines()
    
    # Parse each line as a JSON object and store it in a list
    for line in lines:
        corpus.append(json.loads(line))

In [ ]:
unprocessed_corpus = [doc for doc in corpus]
len(unprocessed_corpus)

171332

### Get queries for documents

In [ ]:
NUM_SAMPLE_QUERIES = 10

In [ ]:
import time
from collections import defaultdict
import pickle


batch_size = 8

doc_to_queries = defaultdict(list)

def add_queries(ids, queries, num_return_sequences=1):
  for i, q in enumerate(queries):
    doc_to_queries[ids[i]].append(q)
    
last_saved_batch = -1
extended_docs = []

for i in tqdm(range(0, len(unprocessed_corpus), batch_size)):
  texts = [doc["text"] for doc in unprocessed_corpus[i:i+batch_size]]
  ids = [doc["_id"] for doc in unprocessed_corpus[i:i+batch_size]]
 
  #Generates 10 queries per document by using random sampling
  sampled_queries = predict(texts, do_sample=True, 
                            num_return_sequences=NUM_SAMPLE_QUERIES)
  ids_list = []
  for id in ids:
    ids_list += [id]*NUM_SAMPLE_QUERIES

  add_queries(ids_list, sampled_queries, num_return_sequences=NUM_SAMPLE_QUERIES)    


 93%|█████████▎| 19830/21417 [6:26:34<31:45,  1.20s/it]

In [ ]:
len(doc_to_queries)

171332

In [ ]:
n_files = 3
json_batch_size = len(doc_to_queries)//n_files

In [ ]:
json_batch_size

57110

In [ ]:
doc_id_to_text_and_title = {doc["_id"]: (doc["text"], doc["title"]) for doc in corpus}

In [ ]:
extended_docs = []

for doc_id, queries in doc_to_queries.items():
  text, title = doc_id_to_text_and_title[doc_id]
  text += '\n' + '\n'.join(queries)
  extended_docs.append({"id":doc_id, "contents": title + '\n' + text})

In [ ]:
len(extended_docs)

171332

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import pickle

with open(f'{workdir}/unjsoned_10_queries_expand_titles.pickle', 'wb') as f:
  pickle.dump(doc_to_queries, f)

In [ ]:
files = f'{workdir}trec-covid'

In [ ]:
j = len(files)

for i in tqdm(range(0, len(extended_docs), json_batch_size)):
  #filename = f"{main_dir}/collections/trec_covid/json_{j}.json"
  filename = f"{workdir}trec-covid/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    for item in extended_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

  0%|          | 0/4 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp/Deep Learning Aplicado à Sistemas de Busca/Notebooks/trec-covid/json_95.json


 25%|██▌       | 1/4 [00:01<00:04,  1.39s/it]

/content/gdrive/MyDrive/Unicamp/Deep Learning Aplicado à Sistemas de Busca/Notebooks/trec-covid/json_96.json


 50%|█████     | 2/4 [00:02<00:02,  1.32s/it]

/content/gdrive/MyDrive/Unicamp/Deep Learning Aplicado à Sistemas de Busca/Notebooks/trec-covid/json_97.json


100%|██████████| 4/4 [00:04<00:00,  1.03s/it]

/content/gdrive/MyDrive/Unicamp/Deep Learning Aplicado à Sistemas de Busca/Notebooks/trec-covid/json_98.json


## Index building in Pyserini

In [ ]:
json_batch_size = len(corpus)//10

In [ ]:
!pip install jsonlines -q

In [ ]:
import jsonlines

j=0

for i in tqdm(range(0, len(extended_docs), json_batch_size)):
  filename = f"{workdir}trec-covid/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    #for item in all_docs[i:i+json_batch_size]:
    for item in extended_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

100%|██████████| 11/11 [00:04<00:00,  2.60it/s]


In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input  /content/gdrive/MyDrive/Unicamp/ \
  --index indexes/expanded_trec_covid \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-12 21:54:24,962 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-12 21:54:24,964 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-12 21:54:24,964 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-12 21:54:24,964 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp/
2023-04-12 21:54:24,965 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-12 21:54:24,965 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-12 21:54:24,965 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-12 21:54:24,966 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-04-12 21:54:24,966 INFO  [main] index.IndexC

In [ ]:
from evaluate import load

def eval(run_file):
  run = pd.read_csv(run_file, sep="\s+", 
                    names=["query", "q0", "docid", "rank", "score", "system"])
  run = run.to_dict(orient="list")

  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10']

In [ ]:
!python -m pyserini.search.lucene \
  --index indexes/expanded_trec_covid \
  --topics trec-covid/queries.tsv \
  --output runs/run.trec-covid.bm25tuned-expanded-trec-covid.txt \
  --hits 1000 \
  --bm25 

Running trec-covid/queries.tsv topics, saving to runs/run.trec-covid.bm25tuned-expanded-trec-covid.txt...
100% 50/50 [00:05<00:00,  8.69it/s]


In [ ]:
run = pd.read_csv("runs/run.trec-covid.bm25tuned-expanded-trec-covid.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [ ]:
results

{'runid': 'Anserini',
 'num_ret': 50000,
 'num_rel': 24673,
 'num_rel_ret': 15332,
 'num_q': 50,
 'map': 0.3691273732512198,
 'gm_map': 0.2836457590400512,
 'bpref': 0.5089901588940219,
 'Rprec': 0.4268886478772377,
 'recip_rank': 0.8839365079365079,
 'P@5': 0.7759999999999999,
 'P@10': 0.7519999999999999,
 'P@15': 0.7133333333333334,
 'P@20': 0.7040000000000001,
 'P@30': 0.6786666666666666,
 'P@100': 0.5988,
 'P@200': 0.5316,
 'P@500': 0.40832,
 'P@1000': 0.30664,
 'NDCG@5': 0.7332192553272483,
 'NDCG@10': 0.7067607039919364,
 'NDCG@15': 0.6722652565815601,
 'NDCG@20': 0.6593713197386752,
 'NDCG@30': 0.6322827652350274,
 'NDCG@100': 0.5543139583418398,
 'NDCG@200': 0.5105889447968706,
 'NDCG@500': 0.5240031776148372,
 'NDCG@1000': 0.6376008973919786}